In [2]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.interpolate import griddata
from IPython.display import clear_output
import time
import imageio

In [14]:
class Chromosome():
    def __init__(self, x=None, y=None):
        if x is None:
            self.gene_x = np.random.uniform(low = 0, high = 100)
        else:
            self.gene_x = x

        if y is None:
            self.gene_y = np.random.uniform(low = 0, high = 100)
        else:
            self.gene_y = y

        self.compute_fitness()

    def compute_fitness(self):
        self.fitness = (1 - self.gene_x)**2 + 1335 * (self.gene_y - self.gene_x**2)**2 #(((np.sin(self.gene_x))**2)+((np.sin(self.gene_y))**2))**(np.exp(self.gene_x+self.gene_y))


    def modify(self, new_x=None, new_y=None):
        if new_x is not None:
            self.gene_x = new_x

        if new_y is not None:
            self.gene_y = new_y
        
        self.fitness = self.compute_fitness()

    def get_genes(self):
        return self.gene_x, self.gene_y

class Population():
    def __init__(self, population_size, crossover_rate, mutation_rate, elitism_size = 0):
        self.population = list()

        self.population_size  = population_size
        self.crossover_rate   = crossover_rate
        self.mutation_rate    = mutation_rate
        self.elitism_size     = elitism_size
        
        for pop in range(population_size):
            self.population.append(Chromosome())

    def mutation(self, chromosome):

        x, y = None, None

        if self.mutation_rate <= np.random.uniform(0, 1):
            x = np.random.uniform(low = 0, high = 100)

        if self.mutation_rate <= np.random.uniform(0, 1):
            y = np.random.uniform(low = 0, high = 100)

        chromosome.modify(x, y)

        return chromosome

    def crossover(self, parent_a, parent_b):

        new_creature_a = Chromosome()
        new_creature_b = Chromosome()

        parent_aX, parent_aY = parent_a.gene_x, parent_a.gene_y 
        parent_bX, parent_bY = parent_b.gene_x, parent_b.gene_y
        new_creature_a.modify(parent_aX, parent_bY)
        new_creature_b.modify(parent_bX, parent_aY)

        return new_creature_a, new_creature_b

    def update_population(self, new_members):
        combined_population = self.population + new_members

        combined_population.sort(key=lambda chromosome: chromosome.fitness)

        #self.population = combined_population[ len(combined_population) - self.population_size - self.elitism_size:]
        self.population = combined_population[ :self.population_size - self.elitism_size]
    
    def best(self, min = True):
        return self.population[0].fitness

    def mean_fitness(self):
        return np.mean([pop.fitness for pop in self.population])

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def plot(population, epoch, best_fitness, mean_fitness, limit = [1000, 500, 200]):
    def rosenbrock(x, y):
        return (1 - x) ** 2 + 1335 * (y - x ** 2) ** 2

    x = []
    y = []
    z = []

    for chr in population:
        x.append(chr.gene_x)
        y.append(chr.gene_y)
        z.append(chr.fitness)

    x = np.array(x)
    y = np.array(y)
    z = np.array(z)

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

    # Define the center coordinates and radii of the circles
    outer_centers = [
        (10, 10), (30, 10), (50, 10), (70, 10), (90, 10),
        (10, 30), (30, 30), (50, 30), (70, 30), (90, 30),
        (10, 50), (30, 50), (50, 50), (70, 50), (90, 50),
        (10, 70), (30, 70), (50, 70), (70, 70), (90, 70),
        (10, 90), (30, 90), (50, 90), (70, 90), (90, 90)

    ]
    outer_radii = 5 * [10, 10, 10, 10, 10]

    inner_radii = [8, 6, 4, 2]

    # Plot the outer circles
    for center, radius in zip(outer_centers, outer_radii):
        outer_circle = plt.Circle(center, radius, edgecolor='black', facecolor='none')
        ax1.add_artist(outer_circle)

    # Plot the inner circles
    for pos in range(len(outer_centers)):
        for posi in range(len(inner_radii)):
            inner_circle = plt.Circle(outer_centers[pos], inner_radii[posi], edgecolor='black', facecolor='none')
            ax1.add_artist(inner_circle)

    # Plot the points
    sc = ax1.scatter(x, y, c=z, cmap='viridis', edgecolor='black')
    cbar = plt.colorbar(sc)

    # Add a line at (1, 1)
    ax1.axvline(x=1, color='red', linestyle='--')
    ax1.axhline(y=1, color='red', linestyle='--')

    # Set the x-axis and y-axis limits
    ax1.set_xlim(0, 100)
    ax1.set_ylim(0, 100)

    # Set axis labels and title
    ax1.set_xlabel('X')
    ax1.set_ylabel('Y')
    ax1.set_title(f'Genetic algorithm Simple: Epoch {epoch}')

    # Plot max and average values over epochs
    ax2.plot(range(epoch + 1), best_fitness, label='Best')
   # ax2.plot(range(epoch + 1), mean_fitness, label='Average')

    # Set axis labels and title for the second plot
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Fitness')
    ax2.set_title('Best and Average Fitness over Epochs')

    # Add legend
    ax2.legend()

    # Display the plot
    plt.tight_layout()
    #plt.plot()

    plt.savefig(f'./img/img_{epoch}.png', 
                transparent = False,  
                facecolor = 'white'
               )
    plt.close()

In [15]:
#Run The Model

#@title Genetic Algorithm
epoch = 20 #@param {type:"integer"}

population_size = 100 #@param {type:"integer"}
crossover_rate = 0.8 #@param {type:"number"}
mutation_rate = 0.1 #@param {type:"number"}

creatures     = Population(population_size, crossover_rate, mutation_rate)
best_fitness  = []
avg_fitness   = []
random_crossover = False #@param {type:"boolean"}
mutation_on   = False #@param {type:"boolean"}
crossover_on  = True #@param {type:"boolean"}
random_pop    = False #@param {type:"boolean"}
for epc in range(epoch):
    new_members = list()

    all_fitness = [creature.fitness for creature in creatures.population]
    prob_from_fitness = all_fitness/np.sum(all_fitness)

    if not random_pop:
        for creature in creatures.population:
            offspring_a, offspring_b = None, None
            if mutation_on:
                #Mutation Phase
                parent_a = creatures.mutation(Chromosome(creature.gene_x, creature.gene_y))
            else:
                parent_a = creature

            if crossover_on:
                #Crossover Phase
                parent_b = creature
                while parent_b == creature:
                    if not random_crossover:
                        parent_b = np.random.choice(creatures.population, size=1,  p=prob_from_fitness)
                    else:
                        parent_b = np.random.choice(creatures.population, size=1)

                offspring_a, offspring_b = creatures.crossover(parent_a, parent_b)

                offspring_a.compute_fitness()
                offspring_b.compute_fitness()
                new_members.append(offspring_a)
                new_members.append(offspring_b)

            if mutation_on and (not crossover_on):
                parent_a.compute_fitness()
                new_members.append(parent_a)
    else:
        new_members = Population(population_size = 20, crossover_rate = 0.8, mutation_rate = 0.1)

    creatures.update_population(new_members)
    best_fitness.append(creatures.best())
    avg_fitness.append(creatures.mean_fitness())

    clear_output(wait=True)
    plot(creatures.population, epc, best_fitness, avg_fitness)

frames = []
for t in range(epoch):
    image = imageio.v2.imread(f'./img/img_{t}.png')
    frames.append(image)


name = "random_as_fuck"

if not random_pop:
    if mutation_on and (not crossover_on):
        name = "only_mutation"
    elif (not mutation_on) and (crossover_on):
        name = "only_crossover_random"
        if (not random_crossover):
            name = "only_crossover_prob"
    elif (mutation_on) and (crossover_on):
        name = "cross_mut_random"
        if (not random_crossover):
            name = "cross_mut_prob"

imageio.mimsave(f'./{name}.gif', 
                frames,          
                duration = 1000)         

for epc in range(epoch):
    print(f"Epoch {epc}: Best Fitness: {best_fitness[epc]}, Average Fitness: {avg_fitness[epc]}")

AttributeError: 'numpy.ndarray' object has no attribute 'gene_x'